### Stockfish vs Stockfish

In [ ]:
import autorootcwd
import chess.engine
import chess.pgn

from src.utils.chess_utils import ChessBoard


def stockfish_vs_stockfish(stockfish_path: str, cpu_nums: int = 4, time_limit: float = 5.0):
    """Play a game between two Stockfish instances.

    Args:
        stockfish_path (str): Path to the Stockfish executable.
        cpu_nums (int, optional): Number of CPUs to be used by Stockfish. Defaults to 4.
        time_limit (float, optional): Time in seconds each engine has to make a move. Defaults to 5.0.

    Returns:
        Game: A chess.pgn.Game object containing the game moves and results.
    """
    board = chess.Board()
    engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

    # Configure Stockfish to use more threads for increased speed
    engine.configure({"Threads": cpu_nums})

    game = chess.pgn.Game()
    node = game

    while not board.is_game_over():
        result = engine.play(board, chess.engine.Limit(time=time_limit))
        move = result.move
        board.push(move)
        ChessBoard().show(board, False)

        node = node.add_variation(move)

    engine.quit()

    # Save the game result to the PGN's headers
    game.headers["Result"] = board.result()
    game.headers["White"] = "Stockfish"
    game.headers["Black"] = "Stockfish"

    # Save to a PGN file
    with open("stockfish_vs_stockfish.pgn", "w") as pgn_file:
        exporter = chess.pgn.FileExporter(pgn_file)
        game.accept(exporter)

    return game


# Example usage:
stockfish_path = "engine/stockfish/stockfish-ubuntu-x86-64-avx2"
stockfish_vs_stockfish(stockfish_path)

### lc0 vs lc0

In [ ]:
import autorootcwd
import chess.engine
import chess.pgn

from src.utils.chess_utils import ChessBoard


def lc0_vs_lc0(lc0_path: str, weights_file: str):
    """Play a game between two Lc0 instances.

    Args:
        lc0_path (str): Path to the Lc0 executable.
        weights_file (str): Path to the weights file for Lc0.

    Returns:
        Game: A chess.pgn.Game object containing the game moves and results.
    """
    board = chess.Board()
    engine = chess.engine.SimpleEngine.popen_uci(lc0_path)

    # Configure Lc0 with the weights file
    engine.configure({"WeightsFile": weights_file})

    game = chess.pgn.Game()
    node = game

    while not board.is_game_over():
        result = engine.play(board, chess.engine.Limit(time=5.0))
        move = result.move
        board.push(move)
        ChessBoard().show(board, False)

        node = node.add_variation(move)

    engine.quit()

    # Save the game result to the PGN's headers
    game.headers["Result"] = board.result()
    game.headers["White"] = "Lc0"
    game.headers["Black"] = "Lc0"

    # Save to a PGN file
    with open("lc0_vs_lc0.pgn", "w") as pgn_file:
        exporter = chess.pgn.FileExporter(pgn_file)
        game.accept(exporter)

    return game


# Example usage:
lc0_path = "engine/Lc0/lc0.exe"
weights_file = "engine/Lc0/client-cache/ac8c69b197421a39d5bd407ff389138d36539b7801b6f7d068d90b16f43e10c0"
lc0_vs_lc0(lc0_path, weights_file)

### lc0 vs Stockfish

In [ ]:
import autorootcwd
import chess.engine
import chess.pgn

from src.utils.chess_utils import ChessBoard


def lc0_vs_stockfish(lc0_path: str, weights_file: str, stockfish_path: str):
    """Play a game between Lc0 and Stockfish.

    Args:
        lc0_path (str): Path to the Lc0 executable.
        weights_file (str): Path to the weights file for Lc0.
        stockfish_path (str): Path to the Stockfish executable.

    Returns:
        Game: A chess.pgn.Game object containing the game moves and results.
    """
    board = chess.Board()
    lc0_engine = chess.engine.SimpleEngine.popen_uci(lc0_path)
    stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

    # Configure Lc0 with the weights file
    lc0_engine.configure({"WeightsFile": weights_file})

    game = chess.pgn.Game()
    node = game

    while not board.is_game_over():
        if board.turn == chess.WHITE:
            result = lc0_engine.play(board, chess.engine.Limit(time=5.0))
        else:
            result = stockfish_engine.play(board, chess.engine.Limit(time=5.0))
        move = result.move
        board.push(move)
        ChessBoard().show(board, False)

        node = node.add_variation(move)

    lc0_engine.quit()
    stockfish_engine.quit()

    # Save the game result to the PGN's headers
    game.headers["Result"] = board.result()
    game.headers["White"] = "Lc0"
    game.headers["Black"] = "Stockfish"

    # Save to a PGN file
    with open("lc0_vs_stockfish.pgn", "w") as pgn_file:
        exporter = chess.pgn.FileExporter(pgn_file)
        game.accept(exporter)

    return game


# Example usage:
lc0_path = "engine/Lc0/lc0.exe"
weights_file = "engine/Lc0/client-cache/ac8c69b197421a39d5bd407ff389138d36539b7801b6f7d068d90b16f43e10c0"
stockfish_path = "engine/stockfish/stockfish-windows-x86-64-avx2.exe"
lc0_vs_stockfish(lc0_path, weights_file, stockfish_path)